In [1]:
import warnings

import pandas as pd

from MLWrappers import SklearnBlackBox
from PrivacyAttacks import MiaPrivacyAttack, LabelOnlyPrivacyAttack, AloaPrivacyAttack
from MLPrivacyEvaluator import PrivacyEvaluator

warnings.simplefilter("ignore", UserWarning)

In [2]:
DS_NAME = 'gaussian'
DATA_FOLDER = f'./data/{DS_NAME}'

# we load the target black box model using our wrapper
target = SklearnBlackBox(f'./models/rf_{DS_NAME}.pkl')

# We load the data used to train, test of the model, as well as the shadow data
train_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_train_set.csv', skipinitialspace=True)
test_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_test_set.csv', skipinitialspace=True)
shadow_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_shadow_set.csv', skipinitialspace=True)

In [3]:
# We initialise the attacks, with the desired parameters for each
mia = MiaPrivacyAttack(target, n_shadow_models=5)
label_only = LabelOnlyPrivacyAttack(target, n_shadow_models=1)
aloa = AloaPrivacyAttack(target, n_shadow_models=1, n_noise_samples_fit=200,
                         shadow_test_size=0.3, undersample_attack_dataset=True)
attacks = [mia, label_only, aloa]

In [4]:
# We initialise the PrivacyEvaluator object
# We pass the target model and the attacks we want to use
evaluator = PrivacyEvaluator(target, attacks)

In [5]:
# We use the fit() method to execute the attacks, starting from the shadow data
evaluator.fit(shadow_set)

Each shadow model uses 100.000 % of the data


100%|██████████| 8000/8000 [02:15<00:00, 59.02it/s]


[2.   1.   0.99 0.98 0.97 0.96 0.95 0.94 0.93 0.92 0.91 0.9  0.89 0.88
 0.87 0.86 0.85 0.84 0.83 0.82 0.81 0.8  0.79 0.78 0.77 0.76 0.75 0.74
 0.73 0.72 0.71 0.7  0.69 0.68 0.67 0.66 0.65 0.64 0.63 0.62 0.61 0.59
 0.58 0.57 0.56 0.55 0.54 0.53 0.49 0.48 0.47 0.46 0.45 0.44 0.4  0.38
 0.36 0.  ]
Each shadow model uses 100.000 % of the data


100%|██████████| 4800/4800 [01:26<00:00, 55.41it/s]

[2.    1.    0.995 0.99  0.985 0.98  0.975 0.97  0.965 0.96  0.955 0.95
 0.945 0.94  0.93  0.925 0.92  0.915 0.91  0.905 0.9   0.895 0.89  0.885
 0.88  0.87  0.865 0.86  0.855 0.83  0.825 0.82  0.8   0.78  0.76  0.75
 0.74  0.73  0.715 0.64  0.63  0.615 0.61  0.58  0.55  0.54  0.505 0.485
 0.   ]


In [6]:
# Then we can obtain the performances using the report() method
results = evaluator.report(train_set, test_set)
print(results)


100%|██████████| 32000/32000 [07:43<00:00, 69.01it/s]


{'mia_attack': {'classification_report': {'IN': {'precision': 0.8074161534288541, 'recall': 0.988359375, 'f1-score': 0.8887718004109806, 'support': 25600}, 'OUT': {'precision': 0.55052790346908, 'recall': 0.05703125, 'f1-score': 0.1033555146538298, 'support': 6400}, 'accuracy': 0.80209375, 'macro avg': {'precision': 0.6789720284489671, 'recall': 0.5226953125, 'f1-score': 0.4960636575324052, 'support': 32000}, 'weighted avg': {'precision': 0.7560385034368993, 'recall': 0.80209375, 'f1-score': 0.7316885432595505, 'support': 32000}}}, 'label_only_attack': {'classification_report': {'IN': {'precision': 0.7996486605182257, 'recall': 0.355625, 'f1-score': 0.4923076923076924, 'support': 25600}, 'OUT': {'precision': 0.1998059665292263, 'recall': 0.64359375, 'f1-score': 0.30494169905607993, 'support': 6400}, 'accuracy': 0.41321875, 'macro avg': {'precision': 0.499727313523726, 'recall': 0.499609375, 'f1-score': 0.3986246956818862, 'support': 32000}, 'weighted avg': {'precision': 0.6796801217204

In [7]:
results['mia_attack']

{'classification_report': {'IN': {'precision': 0.8074161534288541,
   'recall': 0.988359375,
   'f1-score': 0.8887718004109806,
   'support': 25600},
  'OUT': {'precision': 0.55052790346908,
   'recall': 0.05703125,
   'f1-score': 0.1033555146538298,
   'support': 6400},
  'accuracy': 0.80209375,
  'macro avg': {'precision': 0.6789720284489671,
   'recall': 0.5226953125,
   'f1-score': 0.4960636575324052,
   'support': 32000},
  'weighted avg': {'precision': 0.7560385034368993,
   'recall': 0.80209375,
   'f1-score': 0.7316885432595505,
   'support': 32000}}}

In [8]:
results['aloa_attack']['classification_report']

{'IN': {'precision': 0.8000438294408616,
  'recall': 0.9982421875,
  'f1-score': 0.8882207778666019,
  'support': 25600},
 'OUT': {'precision': 0.22413793103448276,
  'recall': 0.00203125,
  'f1-score': 0.004026014245896562,
  'support': 6400},
 'accuracy': 0.799,
 'macro avg': {'precision': 0.5120908802376722,
  'recall': 0.50013671875,
  'f1-score': 0.44612339605624923,
  'support': 32000},
 'weighted avg': {'precision': 0.6848626497595858,
  'recall': 0.799,
  'f1-score': 0.7113818251424608,
  'support': 32000}}

In [9]:
aloa.attack_model.threshold

0.61